In [2]:
from pynq import Overlay, allocate
import numpy as np

overlay = Overlay("test.bit")

# Get the DMA instance for read and write
dma_read = overlay.axi_dma_0  # HP0 (sending data to PL)
dma_write = overlay.axi_dma_1  # HP1 (receiving data from PL)

# Number of integers to send (change this to test different lengths)
num_integers = 10

# Allocate memory for input and output buffers
input_buffer = allocate(shape=(num_integers,), dtype=np.uint32)
output_buffer = allocate(shape=(1,), dtype=np.uint32)  # Single output for max value

# Fill the input buffer with some integers to test max finder
input_data = np.array([1, 8, 3, 7, 4, 10, 2, 6, 5, 9], dtype=np.uint32)
input_buffer[:] = input_data

# Initialize the output buffer with zeros (optional)
output_buffer[:] = np.zeros(1, dtype=np.uint32)

print("Input Buffer: ", input_buffer)

# Start the DMA send through HP0 (from PS to PL)
dma_read.sendchannel.transfer(input_buffer)

# Start the DMA receive through HP1 (from PL to PS)
dma_write.recvchannel.transfer(output_buffer)

# Wait for both send and receive transfers to complete
dma_read.sendchannel.wait()
dma_write.recvchannel.wait()

# Print the maximum value received from the IP
print("Maximum Value Received from IP: ", output_buffer[0])

# Check if the output is correct
expected_max = np.max(input_buffer)
if output_buffer[0] == expected_max:
    print("Test Passed! The maximum value is correct.")
else:
    print("Test Failed! Expected:", expected_max, "but got:", output_buffer[0])

# Clean up the buffers
input_buffer.close()
output_buffer.close()


Input Buffer:  [ 1  8  3  7  4 10  2  6  5  9]
Maximum Value Received from IP:  10
Test Passed! The maximum value is correct.
